In [2]:
%%time
# Expiry, symbol and option info extracted from NSE
# Takes 10 minutes

import requests
import lxml.html as lh
import datetime as dt
import pandas as pd
import numpy as np
import csv
import os

#******         Paths and variables         ****
#_______________________________________________

datapath = r'./zdata/'

#******   Error catch in list comprehension  ****
#________________________________________________

def catch(func, handle=lambda e : e, *args, **kwargs):
    '''List comprehension error catcher'''
    try:
        return func(*args, **kwargs)
    except Exception as e:
        pass

#******               Symbols list          ****
#________________________________________________

eq_symbols='ACC,ADANIENT,ADANIPORTS,ADANIPOWER,AJANTPHARM,ALBK,AMARAJABAT,AMBUJACEM,ANDHRABANK,APOLLOHOSP,\
APOLLOTYRE,ARVIND,ASHOKLEY,ASIANPAINT,AUROPHARMA,AXISBANK,BAJAJ-AUTO,BAJFINANCE,BAJAJFINSV,\
BALKRISIND,BALRAMCHIN,BANKBARODA,BANKINDIA,BATAINDIA,BEML,BERGEPAINT,BEL,BHARATFIN,BHARATFORG,\
BPCL,BHARTIARTL,INFRATEL,BHEL,BIOCON,BOSCHLTD,BRITANNIA,CADILAHC,CANFINHOME,CANBK,CAPF,CASTROLIND,\
CEATLTD,CENTURYTEX,CESC,CGPOWER,CHENNPETRO,CHOLAFIN,CIPLA,COALINDIA,COLPAL,CONCOR,CUMMINSIND,DABUR,\
DALMIABHA,DCBBANK,DHFL,DISHTV,DIVISLAB,DLF,DRREDDY,EICHERMOT,ENGINERSIN,EQUITAS,ESCORTS,EXIDEIND,\
FEDERALBNK,GAIL,GLENMARK,GMRINFRA,GODFRYPHLP,GODREJCP,GODREJIND,GRANULES,GRASIM,GSFC,HAVELLS,HCLTECH,\
HDFCBANK,HDFC,HEROMOTOCO,HEXAWARE,HINDALCO,HCC,HINDPETRO,HINDUNILVR,HINDZINC,ICICIBANK,ICICIPRULI,IDBI,\
IDEA,IDFCBANK,IDFC,IFCI,IBULHSGFIN,INDIANB,IOC,IGL,INDUSINDBK,INFIBEAM,INFY,INDIGO,IRB,ITC,JISLJALEQS,\
JPASSOCIAT,JETAIRWAYS,JINDALSTEL,JSWSTEEL,JUBLFOOD,JUSTDIAL,KAJARIACER,KTKBANK,KSCL,KOTAKBANK,KPIT,\
L%26TFH,LT,LICHSGFIN,LUPIN,M%26MFIN,MGL,M%26M,MANAPPURAM,MRPL,MARICO,MARUTI,MFSL,MINDTREE,MOTHERSUMI,\
MRF,MCX,MUTHOOTFIN,NATIONALUM,NBCC,NCC,NESTLEIND,NHPC,NIITTECH,NMDC,NTPC,ONGC,OIL,OFSS,ORIENTBANK,PAGEIND,\
PCJEWELLER,PETRONET,PIDILITIND,PEL,PFC,POWERGRID,PTC,PNB,PVR,RAYMOND,RBLBANK,RELCAPITAL,RCOM,RELIANCE,\
RELINFRA,RPOWER,REPCOHOME,RECLTD,SHREECEM,SRTRANSFIN,SIEMENS,SREINFRA,SRF,SBIN,SAIL,STAR,SUNPHARMA,SUNTV,\
SUZLON,SYNDIBANK,TATACHEM,TATACOMM,TCS,TATAELXSI,TATAGLOBAL,TATAMTRDVR,TATAMOTORS,TATAPOWER,TATASTEEL,TECHM,\
INDIACEM,RAMCOCEM,SOUTHBANK,TITAN,TORNTPHARM,TORNTPOWER,TV18BRDCST,TVSMOTOR,UJJIVAN,ULTRACEMCO,UNIONBANK,\
UBL,MCDOWELL-N,UPL,VEDL,VGUARD,VOLTAS,WIPRO,WOCKPHARMA,YESBANK,ZEEL'.split(',')

idx_symbols=['NIFTY', 'BANKNIFTY']

#......      Data Limiter   .......
# symbols = eq_symbols+idx_symbols
symbols = eq_symbols+idx_symbols
#..................................

# generic url for equity and index options
url_base = "https://nseindia.com/live_market/dynaContent/live_watch/option_chain/optionKeys.jsp?symbol="
url_end = '&date='

def get_opt_data(symbol):
    '''Gets the options data
    Arg: (symbol) as str
    Returns: scraped dataframe'''
    
#     symbol = 'NIFTY'
    url = url_base + symbol
    
    while_flag = True   # Flag to loop over expiries for a symbol
    while_counter = 0   # Counter for expiry
    expiry = ''         # Initialize expiry for first run
    df = pd.DataFrame() # Initialize return dataframe

    while while_flag:

        # If it is not first run, append next expiry
        if expiry != '':  # This is not the first run
            try:
                expiry = expiry[while_counter]
                url = url_base+symbol+url_end+expiry  # Check to see if this works
            except IndexError:
                while_flag = True   # Get out of the while loop
                break

        # Get raw data from the url
        page = requests.get(url)
        doc = lh.fromstring(page.content)

        #...    Get the Expiries ...#
        #____________________________

        the_path= "//form"   # Get the form content. There are two forms
        expiries = doc.xpath(the_path)[0].text_content()
        strip_chars = [ord('\n'), ord('\xa0'), ord('\t'), ord('\r')] # characters to be stripped
        char_table = {s: ' ' for s in strip_chars} # table for translate to locate the chars

        sym_exp = expiries.translate(char_table).split() # split converts the translated dict into list

        # Get the expiries only, from the table
        expiry = [sym_exp[k+i] 
         for k, v in enumerate(sym_exp) 
         if v in 'Select' 
         for i in range(len(sym_exp) - k)][1:]

        # Convert expiry to datetime
        expiry_dt = [dt.datetime.strptime(date, "%d%b%Y").date() for date in expiry]

        #...   Get the Options Data .... #
        #_________________________________

        tbl_path = "//*[@id='octable']"  # xpath for the table

        # The headers
        opt_head = ['cOI', 'cChnginOI', 'cVolume', 'cIV', 'cLTP', 
                    'cNetChng', 'cBidQty', 'cBidPrice', 'cAskPrice', 'cAskQty', 'Strike', 
                    'pBidQty', 'pBidPrice', 'pAskPrice', 'pAskQty', 'pNetChng', 'pLTP', 
                    'pIV', 'pVolume', 'pChnginOI', 'pOI']

        # Extract the rows within the header in the table
        opt_data = [tr.text_content() for table in doc.xpath(tbl_path) for tr in table[1:]]

        df1 = pd.DataFrame([d.split() for d in opt_data], columns=opt_head)

        df2 = df1.replace(',', '', regex=True) # Remove comma from numbers
        df2 = df2.apply(pd.to_numeric, errors='coerce') # Convert to numeric
        df2.insert(0, 'Expiry', expiry_dt[while_counter]) # Insert the Expiry column
        df2.insert(0, 'Symbol', symbol) # Insert the Symbol

        # Rearrange the columns
        cols_beginning = ['Symbol', 'Expiry', 'Strike']
        df2 = df2[cols_beginning + [c for c in df2 if c not in cols_beginning]]
        while_counter = while_counter + 1
        df = df.append(df2)
        return df

# %%time
nse_options = [catch(lambda: get_opt_data(symbol)) for symbol in symbols]

df_options = pd.concat(nse_options).reset_index(drop=True)

#****        Closing tasks     ****
#___________________________________

# write list of successful symbols to a csv file - for the next program

def write_csv(the_list, filenm):
    '''Writes to the file as a csv
    Args: 
       (the_list) as list
       (filenm) as the name of file, with or without extension
    Returns: None'''
    
    # Check if file_name has an extension
    filenm = filenm.split('.')[0]
    
    if os.path.exists(r'./zdata/'+filenm+r'.csv'):
        print(filenm+'.csv'+' exists and is overwritten!')
    
    with open(datapath+filenm+r'.csv', 'w', newline='') as csvfile:
        wr = csv.writer(csvfile)
        wr.writerow(the_list)

good_idx = list(set(list(df_options.Symbol.unique())) & set(idx_symbols))
good_equity = list(set(list(df_options.Symbol.unique())) & set(eq_symbols))

write_csv(filenm='nse_idx_symbols', the_list=good_idx)
write_csv(filenm='nse_eq_symbols', the_list=good_equity)

# write to pickle file - for the next program   
df_options.to_pickle('./zdata/nse_options.pkl')   # Pickle the dataframe for later use

nse_idx_symbols.csv exists and is overwritten!
nse_eq_symbols.csv exists and is overwritten!
Wall time: 1min 19s


In [3]:
df_options

,Symbol,Expiry,Strike,cOI,cChnginOI,cVolume,cIV,cLTP,cNetChng,cBidQty,...,pBidQty,pBidPrice,pAskPrice,pAskQty,pNetChng,pLTP,pIV,pVolume,pChnginOI,pOI
0,ACC,2018-11-29,1160.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2800.0,1.35,12.90,3600.0,NaN,NaN,NaN,NaN,NaN,NaN
1,ACC,2018-11-29,1180.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3600.0,3.05,9.65,6000.0,NaN,NaN,NaN,NaN,NaN,NaN
2,ACC,2018-11-29,1200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,800.0,7.00,8.05,400.0,-0.70,8.05,35.39,4.0,1600.0,1600.0
3,ACC,2018-11-29,1220.0,NaN,NaN,NaN,NaN,NaN,NaN,2800.0,...,400.0,9.05,15.80,6800.0,2.95,13.00,37.52,1.0,400.0,800.0
4,ACC,2018-11-29,1240.0,NaN,NaN,NaN,NaN,NaN,NaN,2000.0,...,2000.0,10.35,18.05,6800.0,NaN,15.00,NaN,NaN,NaN,400.0
5,ACC,2018-11-29,1260.0,NaN,NaN,NaN,NaN,NaN,NaN,4000.0,...,5600.0,13.15,16.00,400.0,-1.65,13.15,30.18,18.0,5600.0,6800.0
6,ACC,2018-11-29,1280.0,NaN,NaN,NaN,NaN,NaN,NaN,7600.0,...,2000.0,18.70,21.00,800.0,7.95,21.00,33.40,3.0,1200.0,1200.0
7,ACC,2018-11-29,1300.0,NaN,NaN,NaN,NaN,NaN,NaN,4800.0,...,400.0,25.50,28.90,800.0,3.80,25.50,31.51,19.0,4400.0,10800.0
8,ACC,2018-11-29,1320.0,NaN,NaN,NaN,NaN,NaN,NaN,6400.0,...,1200.0,31.95,33.95,800.0,5.70,33.00,31.76,43.0,4000.0,6400.0
9,ACC,2018-11-29,1340.0,400.0,400.0,1.0,21.81,47.85,-282.00,800.0,...,7200.0,38.25,43.35,800.0,5.75,41.70,31.99,22.0,2800.0,6400.0
